# Module 3 - Exercise 1: Build Custom MLP

## Learning Objectives
- Design and implement custom neural network architectures
- Master activation functions and their properties
- Implement weight initialization strategies
- Add regularization techniques (dropout, batch norm)
- Compare different architectural choices

*Complete MLP implementation from scratch with modern techniques.*